# 1. Define the Populations
First, we use `src\data\populations\users.py` to define the Population object. Here, we 
1. provide a **raw** input file containing some information about the users (for example, some kind of demographic information as birthday),
2. preprocess and filter users based on soem criteria (for example, you want to exclude people beyond certain age),
3. create data splits (train, val, test).

As you create the object and call specific methods, the Population object runs all these processes and saves outputs in `data\processed\populations` folder. 

### Why do we need the Populations object?

It defines cohort of users that we want to work with.
Based on the populations, we further create a dataset that contains specific cohorts of people. You can define multiple populations and create various datasets based on various specifications of users.

In [ ]:
# Define the Population object
from src.dataloaders.population.users import UserPopulation
users = UserPopulation()
## run the preprocessing part
users.population()
## create datasplits 
users.data_split()
## You can also just run the prepare function to do both of the above steps
# users.prepare() 

After you run these commands for the first time, you will see that the results are saved in `data\procesed\populations`. 
Next time you the same functions, instead of calculating everything, the object would read the data from the `data\processed\populations` folder. **This is important for very big datasets**.

If you want to redo the calculations, you need to empty the `data\processed\populations` folder. It also applies to cases, when you change the code of the `src\data\populations\users.py`, as the Population object saves `arguments` so it can validate that you call a specific version of the Population object. 

# 2. Define the Source
The Source objects in `src\data\sources` specify how to process a specific *source* of data (for example, the dummy labor dataset). Inside of this object we specify how to preprocess and tokenie the dataset. 

In [ ]:
from src.dataloaders.sources.synth_labor import SyntheticLaborSource

In [2]:
## How to create a SOURCE!
r = SyntheticLaborSource().prepare()